# Sanic 集成 GraphQL(Strawberry)

参考：[Sanic GraphQL](https://strawberry.rocks/docs/integrations/sanic)。

[Strawberry](https://pypi.org/project/strawberry-graphql/) 带有基本的 Sanic 集成。它提供了视图来服务你的 GraphQL 模式：

```python
from strawberry.sanic.views import GraphQLView


from api.schema import Schema
 
app = Sanic(__name__)


app.add_route(
    GraphQLView.as_view(schema=schema, graphiql=True),
    "/graphql",
)
```

## `GraphQLView` 选项

`GraphQLView` 目前接受选项：

- `schema`：强制性的，由 {class}`strawberry.Schema` 创建的模式。
- `graphiql`：可选，默认为 `True`，是否启用 GraphiQL 接口。
- `allow_queries_via_get`：可选，默认为 `True`，是否通过 GET 请求启用查询。
- `def encode_json(self, data: GraphQLHTTPResponse) -> str`。

```{tip}
参考[部署](https://strawberry.rocks/docs/operations/deployment)。

GraphiQL 在测试和开发期间很有用，但默认情况下应在生产中禁用。
```

## 扩展视图

基本的 GraphQLView 类可以通过覆盖以下方法来扩展：

- `async get_context(self) -> Any`
- `get_root_value(self) -> Any`
- `process_result(self, result: ExecutionResult) -> GraphQLHTTPResponse`

### `get_context`

通过重写 `GraphQLView.get_context` 可以为你的解析器提供自定义上下文对象。你可以在这里返回任何东西；默认情况下 GraphQLView 返回带带有请求的字典。

```python
class MyGraphQLView(GraphQLView):
    async def get_context(self, request) -> Any:
        return {"example": 1}

@strawberry.type
class Query:
    @strawberry.field
    def example(self, info: Info) -> str:
        return str(info.context["example"])
```

### `get_root_value`

通过重写 `GraphQLView.get_root_value` 为您的模式提供自定义根值。这可能不常用，但在某些情况下可能有用。

```python
class MyGraphQLView(GraphQLView):
    def get_root_value(self) -> Any:
        return Query(name="Patrick")


@strawberry.type
class Query:
    name: str
```

这里配置了 `Query`，其中请求名称字段将通过自定义根值返回 `"Patrick"`。

### `process_result`

通过重写 ` GraphQLView.process_result` 在将结果发送到客户端之前，您可以自定义和/或处理结果。这可以用于记录错误，甚至隐藏错误（例如隐藏内部异常）。

它需要返回 `GraphQLHTTPResponse` 对象并接受执行结果。

```python
from strawberry.sanic.views import GraphQLView
from strawberry.http import GraphQLHTTPResponse, process_result
from strawberry.types import ExecutionResult
from sanic.request import Request
from graphql.error.graphql_error import format_error as format_graphql_error


class MyGraphQLView(GraphQLView):
    async def process_result(
        self, request: Request, result: ExecutionResult
    ) -> GraphQLHTTPResponse:
        if result.errors:
            result.errors = [format_graphql_error(err) for err in result.errors]


        return process_result(data)
```

在本例中，对结果进行默认处理，但可以根据您的需要进行调整。

### `encode_json`

`encode_json` 允许自定义 JSON 响应的编码。默认情况下，使用 `json.dumps`，但您可以重写此方法以使用不同的编码器。

```python
class MyGraphQLView(GraphQLView):
    def encode_json(self, data: GraphQLHTTPResponse) -> str:
        return json.dumps(data, indent=2)
```